In [1]:
#importing libraries for emails
import imaplib
import base64
import email
import re
import getpass #for entering password securely
#importing boto3 module to connect to aws services
import boto3
#importing the following functions to convert the date of different timezone to local timezone
from email.utils import parsedate_tz, mktime_tz, formatdate

In [2]:
#using Amazon s3
s3 = boto3.resource('s3')

In [3]:
#to input email_id and password to access emails
email_user=input('Email:')
email_pass=getpass.getpass(prompt='Password: ', stream=None)

Email:agrawalaastha798@gmail.com
Password: ········


In [4]:
#defining function for uploading only csv files to s3 bucket
def func_for_uploading_file(email_message):
    
    for part in email_message.walk():
        
        if part.get_content_maintype() == 'multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
        
        #finding file names from part
        fileName = part.get_filename()
        #splitting file names into a list
        file_lists=re.split('/n',fileName)

        
        for file in file_lists:
            
            if file.endswith('.csv') or file.endswith('.CSV'): #finding whether file is csv file or not
                print(file)
                # The object does not exist.Therefore,this part deals with uploading of the file in s3.
                
                data = part.get_payload(decode=True)
                
                #to extract date from mail and converting it into localtimzone
                date_time=email_message['Date']
                ts = mktime_tz(parsedate_tz(date_time))
                date_time_local=formatdate(ts, localtime=True)
                
                #extracting date and time from the string using regex
                date = re.findall('\d{2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}', date_time_local)
                time=re.findall("(?:[01]\d|2[0123]):(?:[012345]\d):(?:[012345]\d)",date_time_local)
                
                #uploading the file in s3 bucket
                filename=file+' '+date[0]+' '+time[0]
                #creating directory as filename with date and time of receiving
                directory_name = filename
                s3.Bucket('csvfilesfromemails').put_object(Key=filename+'/'+file, Body=data)
                print('File uploaded successfully.\n')
      
            else:
                print(file)
                print('Unwanted file.\n')
    
  


In [10]:
#making connection with the host over an SSL encrypted socket

mail=imaplib.IMAP4_SSL("imap.gmail.com",993)

In [11]:
#login into the account
'''
if at login time the app password is required, simply replace the account password with application password
at the entrance of password.'''
mail.login(email_user, email_pass)


('OK', [b'agrawalaastha798@gmail.com authenticated (Success)'])

In [12]:
#listing the folders
mail.list()

('OK',
 [b'(\\HasNoChildren) "/" "INBOX"',
  b'(\\HasChildren \\Noselect) "/" "[Gmail]"',
  b'(\\All \\HasNoChildren) "/" "[Gmail]/All Mail"',
  b'(\\Drafts \\HasNoChildren) "/" "[Gmail]/Drafts"',
  b'(\\HasNoChildren \\Important) "/" "[Gmail]/Important"',
  b'(\\HasNoChildren \\Sent) "/" "[Gmail]/Sent Mail"',
  b'(\\HasNoChildren \\Junk) "/" "[Gmail]/Spam"',
  b'(\\Flagged \\HasNoChildren) "/" "[Gmail]/Starred"',
  b'(\\HasNoChildren \\Trash) "/" "[Gmail]/Trash"'])

In [13]:
#to select inbox folder to receive mail
mail.select('Inbox')

('OK', [b'12'])

In [14]:
#to receive unread emails from the account


t, data = mail.search(None, 'UNSEEN')

id_list = data[0].split()
id_list

[b'12']

In [15]:
#to fetch all the unseen mails from inbox

if id_list!=[]:
    for num in id_list:
        #fetching the mail
        typ, data = mail.fetch(num, '(RFC822)' )
        raw_email = data[0][1]


        # converts byte literal to string removing b''
        raw_email_string = raw_email.decode('utf-8')
        email_message = email.message_from_string(raw_email_string)

        #calling the function for uplodaing csv files
        func_for_uploading_file(email_message)
else:
    print("No unseen messages!")
   


MOCK_DATA.csv
File uploaded successfully.

people1.json
Unwanted file.

product_info.csv
File uploaded successfully.



In [16]:
#marking the emails as seen
#we can skip this step as the mails are already flagged as seen when they are fetched using fetch function.
for num in id_list:
        mail.store(num, '+FLAGS', '\Seen')

In [17]:
#closing the connection
mail.close()
mail.logout()

('BYE', [b'LOGOUT Requested'])